In [ ]:
import aocd

data = [ (v[0], int(v[1:])) for v in aocd.get_data(day=1, year=2025).splitlines() ]
part1, part2, cur, pd = 0, 0, 50, "R"

for d,v in data:
    if d != pd: pd, cur = d, (100-cur)%100
    md, cur = divmod(cur+v,100)
    part1, part2 = part1 + (cur == 0), part2 + md
    
print("Part1:",part1,"Part2:",part2)

Part1: 1135 Part2: 6558


In [99]:
import aocd

data = [ v.split("-") for v in aocd.get_data(day=2, year=2025).split(",") ]
part1, part2 = 0, 0

def bad(fro, to, rep):
    min_l = (len(fro)-1)//rep+1
    max_l = len(to)//rep
    if min_l>max_l: return set()
    mi = (10**(min_l-1)) if len(fro)%rep else (int(fro[:min_l]) + (fro[:min_l]*(rep-1)<fro[min_l:]))
    ma = (10**(max_l)-1) if len(to)%rep else (int(to[:max_l]) - (to[:max_l]*(rep-1)>to[max_l:]))
    return {int(str(v)*rep) for v in range(mi,ma+1)}

for fro,to in data:
    s = set(bad(fro,to,2))
    part1 += sum(s)
    for k in range(3,len(to)+1): s.update(bad(fro,to,k))
    part2 += sum(s)

print("Part1:",part1,"Part2:",part2)

sum(len(bad("1","999999999999",k)) for k in range(2,7))


Part1: 28846518423 Part2: 31578210022


1011195

In [31]:
import aocd

data = [ v.split("-") for v in aocd.get_data(day=2, year=2025).split(",") ]

In [41]:
%%timeit

part1, part2 = 0, 0

def bad(fro, to, rep):
    ret = 0
    min_l = (len(fro)-1)//rep+1
    max_l = len(to)//rep
    if min_l>max_l: return ret
    mi = (10**(min_l-1)) if len(fro)%rep else (int(fro[:min_l]) + (fro[:min_l]*(rep-1)<fro[min_l:]))
    ma = (10**(max_l)-1) if len(to)%rep else (int(to[:max_l]) - (to[:max_l]*(rep-1)>to[max_l:]))
    mil = len(str(mi))
    mal = len(str(ma))
    for l in range(mil,mal+1):
        if l == mil: tmi = mi
        else: tmi = 10*(mil-1)
        if l == mal: tma = ma
        else: tma = 10*(mil) - 1
        ret += (tmi+tma)*(tma-tmi+1)//2*sum((10**(k*l)) for k in range(rep))
    return ret

for fro,to in data:
    s = bad(fro,to,2)
    part1 += s
    part2 += s
    for k in [3,5]: part2 += k//abs(k)*bad(fro,to,abs(k))

#print("Part1:",part1,"Part2:",part2)



577 μs ± 53.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
get_ipython().__class__

ipykernel.zmqshell.ZMQInteractiveShell

In [51]:
import aocd

def getjolt(batt,k):
    res, ret, cur, n = batt[-k:], 0, -1, len(batt)
    for i in range(len(batt)):
        while cur>=0 and res[cur]<batt[i] and n-i>=k-cur: cur -= 1
        if cur<k-1: res[cur+1], cur = batt[i], cur+1
    for v in res: ret = ret*10+v
    return ret

input = aocd.get_data(day=3, year=2025)
data = [[int(char) for char in line] for line in input.splitlines()]

print("Part1:",sum(map(lambda x:getjolt(x,2),data)),"Part1:",sum(map(lambda x:getjolt(x,12),data)))

Part1: 17155 Part1: 169685670469164


In [ ]:
import aocd
import numpy as np
input = aocd.get_data(day=4, year=2025)

mt = np.matrix([[c=="@" for c in line] for line in input.splitlines()])
h, w = mt.shape

def removable_rolls(mt):
    t = np.zeros((h,w))
    t[:h-1,:w-1] += mt[1:,1:]
    t[1:,1:] += mt[:h-1,:w-1]

    t[:h-1,:] += mt[1:,:]
    t[1:,:] += mt[:h-1,:]

    t[:,1:] += mt[:,:w-1]
    t[:,:w-1] += mt[:,1:]

    t[:h-1,1:] += mt[1:,:w-1]
    t[1:,:w-1] += mt[:h-1,1:]
    return np.logical_and((t<4),mt)

rm = removable_rolls(mt)
rm_size = np.matrix.sum(rm)
part1 = part2 = rm_size
while rm_size:
    mt = np.logical_xor(rm, mt)
    rm = removable_rolls(mt)
    rm_size = np.matrix.sum(rm)
    part2 += rm_size

print("Part1:",part1,"Part2:",part2)



Part1: 1344 Part2: 8112


In [89]:
import aocd
import numpy as np
input = aocd.get_data(day=4, year=2025)

mat = {(i,j) for i,line in enumerate(input.splitlines()) for j,c in enumerate(line) if c == "@" }

def neb(i,j): return({(i-1,j-1),(i-1,j+1),(i-1,j),(i,j-1),(i,j+1),(i+1,j-1),(i+1,j),(i+1,j+1)})

acc = { p for p in mat if len(neb(*p) & mat)<4 }
part1 = part2 = len(acc)
while len(acc):
    mat -= acc
    acc = { p for p in mat if len(neb(*p) & mat)<4 }
    part2 += len(acc)
    
print("Part1:",part1,"Part2:",part2)


Part1: 1344 Part2: 8112


In [181]:
import aocd

input = aocd.get_data(day=5, year=2025)
import bisect

R,L = input.split("\n\n")
R = sorted((tuple(map(int,r.split("-"))) for r in R.splitlines()))
L = list(map(int,L.splitlines()))

R2 = []
cur = -1
for a,b in R:
    if a <= cur:
        if b>=cur:
            R2[-1] = (R2[-1][0],b)
            cur = b+1
    elif a > cur:
        R2.append((a,b))
        cur = b+1
        
part1 = sum(v>=R2[p][0] for v in L for p in [bisect.bisect_left(R2,v,key=lambda x:x[1])] if p<len(R2))
print("Part1:",part1,"Part2:",sum(b-a+1 for a,b in R2))


Part1: 661 Part2: 359526404143208


In [25]:
ranges = [tuple(map(int, p.split("-"))) for p in aocd.get_data(day=2, year=2025).split(",")]
from math import log10, floor

def num_digits(x: int) -> int:
    return floor(log10(x)) + 1


In [29]:
%%timeit

# Optimized O(log n) version for part 2:

def sum_invalid_ids_pt2(low: int, high: int) -> int:
    
    L = num_digits(low)
    if num_digits(high) > L:
        return sum_invalid_ids_pt2(low, 10**L - 1) + sum_invalid_ids_pt2(10**L, high)
    if L == 1:
        return 0
    
    # Look only at pattern lengths l that divide L and do not have a multiple (duplicates):
    ll = {l for l in range(1, max(1, L // 2 + 1)) if (L % l) == 0}
    ll = {l for l in ll if all(l*i not in ll for i in range(2, max(ll)+1))}

    total = 0
    for l in ll:
        low_q = low // 10 ** (L - l)
        high_q = high // 10 ** (L - l)

        # Consider sequence of values repeated patterns based of low_q ... high_q:
        
        # Treat the first and last values specially
        first = sum(low_q * 10 ** (p * l) for p in range(L // l))
        if low <= first <= high:
            total += first
            
        if low_q < high_q:
            last = sum(high_q * 10 ** (p * l) for p in range(L // l))
            if low <= last <= high:
                total += last
        
            if low_q < high_q - 1:
                # Calculate the sum of the middle values using arithmetic series
                S = ((high_q - low_q - 1) * (high_q + low_q) // 2)
                total += sum(S * 10 ** (p * l) for p in range(L // l))
                
    # treat same-digit numbers (present in all pattern lengths)
    for d in range(1, 10):
        if low <= int(str(d) * L) <= high:
            total -= int(str(d) * L) * (len(ll) - 1)
    

    return total

part2 = sum(sum_invalid_ids_pt2(*r) for r in ranges)

2.13 ms ± 573 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
